In [ ]:
!conda install -c intel scikit-learn

# Training a sentiment analysis classifier based on supervised machine learning algorithms

In [ ]:
import string

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

In [ ]:
stop_words = set(stopwords.words('spanish'))

In [ ]:
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

### Loading labeled tweets

In [ ]:
# Dataset loaded from: https://docs.google.com/spreadsheets/d/11_E2ngsEOyTQwbwVVRHY5urzFF95BQCV/edit#gid=1788161364
tweets_df = pd.read_csv('./data/tweets_labeled.csv', sep = ',')

In [ ]:
tweets_df.head()

In [ ]:
tweets_df['sentiment'].value_counts(dropna = False, normalize = True)

In [ ]:
tweets_df['emotion'].value_counts(dropna = False, normalize = True)

### Leaving out unlabeled texts, this data is not useful for training or validating a supervised model

In [ ]:
tweets_labeled_df = tweets_df.loc[tweets_df['sentiment'].notnull() & tweets_df['sentiment'].isin(['positive', 'negative'])]
#tweets_labeled_df = tweets_df.loc[tweets_df['sentiment'].notnull()]

In [ ]:
tweets_labeled_df.shape

In [ ]:
tweets_nolabeled_df = tweets_df.loc[tweets_df['sentiment'].isnull()]

In [ ]:
tweets_nolabeled_df.shape

### Splitting train and test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets_labeled_df['full_text'], tweets_labeled_df['sentiment'], test_size = 0.2, stratify = tweets_labeled_df['sentiment'], random_state = 1)


In [ ]:
X_train.shape

In [ ]:
pd.Series(y_train).value_counts(normalize = True)

In [ ]:
X_test.shape

In [ ]:
pd.Series(y_test).value_counts(normalize = True)

### Vectorizing texts

<table>
    <tbody>
        <tr>
            <td>
                <h4>Bag of Words</h4>
                <img src="./imgs/bow.png" style="width: 500px;">
            </td>
            <td>
                <h4>TF-IDF</h4>
                <img src="./imgs/tf-idf.png" style="width: 500px;">
            </td>
        </tr>
    </tbody>
</table>

In [ ]:
bow = CountVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [ ]:
tfidf = TfidfVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [ ]:
X_bow = bow.fit_transform(X_train)

In [ ]:
X_tfidf = tfidf.fit_transform(X_train)

### Training and evaluating a model using BOW

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
model.fit(X_bow, y_train)

In [ ]:
y_train_bow_predict = model.predict(X_bow)
y_test_bow_predict = model.predict(bow.transform(X_test))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_bow_predict)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_bow_predict)

In [ ]:
print('Precision:', precision_score(y_test, y_test_bow_predict, pos_label = 'positive'))
print('Recall:', recall_score(y_test, y_test_bow_predict, pos_label = 'positive'))
print('F1:', f1_score(y_test, y_test_bow_predict, pos_label = 'positive'))

#print('Precision:', precision_score(y_test, y_test_bow_predict, average = None))
#print('Recall:', recall_score(y_test, y_test_bow_predict, average = None))
#print('F1:', f1_score(y_test, y_test_bow_predict, average = None))

### Training and evaluating a model using TF-IDF

In [ ]:
logistic_model = LogisticRegression(random_state = 2)

In [ ]:
logistic_model.fit(X_tfidf, y_train)

In [ ]:
y_train_tfidf_predict = logistic_model.predict(X_tfidf)
y_test_tfidf_predict = logistic_model.predict(bow.transform(X_test))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_tfidf_predict)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_tfidf_predict)

In [ ]:
print('Precision:', precision_score(y_test, y_test_tfidf_predict, pos_label = 'positive'))
print('Recall:', recall_score(y_test, y_test_tfidf_predict, pos_label = 'positive'))
print('F1:', f1_score(y_test, y_test_tfidf_predict, pos_label = 'positive'))

### How interpret the results?

<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/confusion-matrix.png" style="height: 400px;">
            </td>
            <td>
                <img src="./imgs/precision-recall.png" style="height: 600px;">
            </td>
        </tr>
    </tbody>
</table>